In [ ]:
#C/P files (not storage friendly..)

cp Microcoleus/Community/CAWBG24/*.fa FengMin/CAWBG24.Pro/


find Mapping/ -type f -name "*JU.bins.fna" -exec rm {} \;
	•	.gff: Contains the genome coordinates and functional annotations.
	•	.faa: Protein sequences in FASTA format.
	•	.fna: DNA ORFs in FASTA format.


In [ ]:
#Prokka Annotation 

cp Microcoleus/Community/CAWBG51/*.fa FengMin/week_3_prokka_pilot/CAWBG51.prok/

# Prokka community .fa files using defualt settings
Nano prokka_pilot.sh



#!/bin/bash -e
#SBATCH -A uoa03265
#SBATCH -J spades_assembly_01_v2
#SBATCH --time 00:30:00
#SBATCH --mem 10GB
#SBATCH --ntasks 1
#SBATCH --cpus-per-task 8

#!/bin/bash


# Loop through each directory (assuming they are in the current working directory)
for dir in */; do
  # Check if the directory exists
  if [ -d "$dir" ]; then
    # Loop through all .fa files in the directory
    for fa_file in "$dir"*.fa; do
      # Extract the filename without the extension
      file_name=$(basename "$fa_file" .fa)
      
      # Run Prokka with the output directory and prefix matching the .fa file name
      prokka --outdir "${dir}${file_name}_output" --prefix "$file_name" "$fa_file"
    done
  fi
done


In [ ]:
# 1. DNA gene calls (fasta)	- from Prokka
# 2. DNA gene +- flanking regions	
# 3. The flanking regions with the gene deleted 

#!/bin/bash
#SBATCH --time=23:59:59
#SBATCH --mem=35GB
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=16
#SBATCH --output=slurm_%j.out
#SBATCH --error=slurm_%j.err

# Optional safety toggle — comment out for debugging
set -euo pipefail

# Loop recursively over all .gff files, excluding .ipynb_checkpoints
find . -type f -name "*.gff" ! -path "*/.ipynb_checkpoints/*" | while read gff_file; do
  echo "Processing file: $gff_file"
  dir=$(dirname "$gff_file")
  base=$(basename "$gff_file" .gff)

  fna_file="$dir/$base.fna"
  if [ ! -f "$fna_file" ]; then
    echo "FASTA file $fna_file does not exist. Skipping."
    continue
  fi

  # Output file names
  gene_fasta="$dir/${base}_gene_only.fa"
  full_fasta="$dir/${base}_gene_with_flanks.fa"
  del_fasta="$dir/${base}_gene_deletion.fa"

  # Clear old contents
  > "$gene_fasta"
  > "$full_fasta"
  > "$del_fasta"

  # Make sure fasta index exists
  samtools faidx "$fna_file"

  # Process each CDS from the GFF
  awk 'BEGIN { FS="\t"; OFS="\t" }
  $3=="CDS" {
    locus = "NA";
    product = "NA";
    attr = $9;
    if (sub(/.*locus_tag=/, "", attr)) {
      locus = attr; sub(/;.*/, "", locus);
    }
    attr = $9;
    if (sub(/.*product=/, "", attr)) {
      product = attr; sub(/;.*/, "", product);
    }
    orig_start = $4;
    orig_end = $5;
    strand = $7;
    full_start = orig_start - 250;
    if (full_start < 1) full_start = 1;
    full_end = orig_end + 250;
    print $1, orig_start, orig_end, full_start, full_end, locus, strand, product;
  }' "$gff_file" | while read seq_id orig_start orig_end full_start full_end locus strand product; do

    # --- Extract gene only ---
    gene_seq=$(samtools faidx "$fna_file" "${seq_id}:${orig_start}-${orig_end}" | tail -n +2 | tr -d '\n')
    if [ "$strand" = "-" ]; then
      gene_seq=$(echo "$gene_seq" | rev | tr 'ACGTacgt' 'TGCAtgca')
    fi

    gene_len=${#gene_seq}
    if [ "$gene_len" -lt 500 ]; then
      continue  # skip short genes
    fi

    # --- Output gene_only ---
    header_gene=">${locus} ${product} | start:${orig_start} stop:${orig_end} | strand:${strand} | gene_only"
    echo "$header_gene" >> "$gene_fasta"
    echo "$gene_seq" >> "$gene_fasta"

    # --- Extract full gene + flanks ---
    full_seq=$(samtools faidx "$fna_file" "${seq_id}:${full_start}-${full_end}" | tail -n +2 | tr -d '\n')
    if [ "$strand" = "-" ]; then
      full_seq=$(echo "$full_seq" | rev | tr 'ACGTacgt' 'TGCAtgca')
    fi
    header_full=">${locus} | start:${orig_start} stop:${orig_end} | strand:${strand} | full_start:${full_start} full_end:${full_end} | product:${product} | gene_with_flanks"
    echo "$header_full" >> "$full_fasta"
    echo "$full_seq" >> "$full_fasta"

    # --- Extract flanks only (simulate deletion) ---
    if [ "$orig_start" -gt "$full_start" ]; then
      upstream=$(samtools faidx "$fna_file" "${seq_id}:${full_start}-$((orig_start - 1))" | tail -n +2 | tr -d '\n')
    else
      upstream=""
    fi

    if [ "$orig_end" -lt "$full_end" ]; then
      downstream=$(samtools faidx "$fna_file" "${seq_id}:$((orig_end + 1))-${full_end}" | tail -n +2 | tr -d '\n')
    else
      downstream=""
    fi

    del_seq="${upstream}${downstream}"
    if [ "$strand" = "-" ]; then
      del_seq=$(echo "$del_seq" | rev | tr 'ACGTacgt' 'TGCAtgca')
    fi
    header_del=">${locus} ${product} | start:${orig_start} stop:${orig_end} | strand:${strand} | flanking_deletion(+/-250bp)"
    echo "$header_del" >> "$del_fasta"
    echo "$del_seq" >> "$del_fasta"

  done
done

In [ ]:
#Readmapping 

#!/bin/bash
#SBATCH --time=48:59:59
#SBATCH --mem=15GB
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8



for dir in 0*/ 10*/ 11*/; do
    # Verify that we are working with a directory.
    [[ -d "$dir" ]] || continue
    # Remove trailing slash from the directory name.
    dir="${dir%/}"
    
    echo "------------------------------"
    echo "Processing directory: $dir"
    
    ###############################
    # Step 1: Concatenate *.fa files from bowtie_fas into JUJU.bins.fna
    if [ -d "$dir/bowtie_fas" ]; then
        if [ -f "$dir/JUJU.bins.fna" ]; then
            echo "  Removing existing $dir/JUJU.bins.fna..."
            rm "$dir/JUJU.bins.fna"
        fi
        echo "  Concatenating files in $dir/bowtie_fas to create JUJU.bins.fna..."
        cat "$dir/bowtie_fas/"*.fa >> "$dir/JUJU.bins.fna"
    else
        echo "  Skipping concatenation: No bowtie_fas directory found in $dir."
    fi

    ###############################
    # Step 2: Build bowtie2 index
    if [ -f "$dir/JUJU.bins.fna" ]; then
        # Ensure the output directory exists.
        mkdir -p "$dir/bin_coverage"
        
        # Remove any existing bowtie2 index files (with names like JUJU.bins.1.bt2, etc.).
        if ls "$dir/bin_coverage/JUJU.bins".*.bt2 1> /dev/null 2>&1; then
            echo "  Removing existing bowtie2 index files in $dir/bin_coverage..."
            rm "$dir/bin_coverage/JUJU.bins".*.bt2
        fi

        echo "  Running bowtie2-build for $dir..."
        bowtie2-build "$dir/JUJU.bins.fna" "$dir/bin_coverage/JUJU.bins"
    else
        echo "  Skipping bowtie2-build: No JUJU.bins.fna file found in $dir."
    fi

    ###############################
    # Step 3: Align reads and process BAM files
    # Extract numeric prefix from the directory name.
    num=$(basename "$dir" | grep -oE '^[0-9]+')
    if [ -z "$num" ]; then
        echo "  Skipping alignment: No numeric prefix found in $dir."
        continue
    fi
    # Format the number to two digits (e.g. 9 -> 09)
    num=$(printf "%02d" "$num")
    
    # Define read files and outputs.
    reads_dir="$dir/Reads"
    fq1="$reads_dir/t.${num}.1.fastq.gz"
    fq2="$reads_dir/t.${num}.2.fastq.gz"
    index="$dir/bin_coverage/JUJU.bins"
    sam_output="$dir/bin_coverage/output_${num}.sam"
    bam_output="$dir/bin_coverage/output_${num}.bam"
    depth_output="$dir/bin_coverage/bins_cov_table_${num}.txt"
    
    # Check for the necessary files: paired-end reads and bowtie2 index (index.1.bt2 must exist).
    if [[ -f "$fq1" && -f "$fq2" && -f "$index.1.bt2" ]]; then
        echo "  Running Bowtie2 alignment for $dir..."
        bowtie2 -x "$index" -1 "$fq1" -2 "$fq2" -S "$sam_output"
        echo "  Bowtie2 alignment completed for $dir."
    
        echo "  Sorting SAM into BAM for $dir..."
        samtools sort -o "$bam_output" "$sam_output"
        echo "  SAM sorting completed for $dir."
    
        echo "  Summarizing BAM contig depths for $dir..."
        jgi_summarize_bam_contig_depths --outputDepth "$depth_output" "$dir/bin_coverage/"*.bam
        echo "  Depth summarization completed for $dir."
    else
        echo "  Skipping alignment and downstream processing for $dir: Missing required files (reads or index)."
    fi

    echo "Finished processing $dir."
done

echo "All processes completed successfully!"



In [ ]:
#Read Depth Visualisation 

#!/bin/bash
#SBATCH --time=48:59:59
#SBATCH --mem=15GB
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8

for f in *.bam; do
  samtools depth "$f" > "${f%.bam}.depth"
done

In [ ]:
#R Studio 
coverage_targets <- read.table(file = "cov_targets.txt", header = FALSE)
depth_file <- "output_08.depth"

data <- read.table(depth_file, header = FALSE, )


for (b in 1:nrow(coverage_targets)){
	x <- vector()
	y <- vector()
	cnt = 1
	for (a in 1:nrow(data)){
		if(data[a,1] == coverage_targets[b,1]){ 
			x[cnt] = data[a,2]
			y[cnt] = data[a,3]
			cnt = cnt + 1
		}	
	}
	out_head <- "bp\tread depth"
	out_file <- paste(coverage_targets[b,1], "bp-depth", sep = ".")
	write(out_head, file = out_file)
	for (z in 1:cnt){
		out_ln <- paste(x[z], y[z], sep = "\t")
		write(out_ln, file = out_file, append = TRUE)
	}
}

in_files <- list.files(pattern = "*.bp-depth")
for (y in 1:length(in_files)){
	filn <- read.table(in_files[y], header = TRUE, sep = "\t")
	pdf_name <- paste(in_files[y], "pdf", sep = ".")
	pdf(pdf_name)
	plot(filn, type = "l", xlab = "coordinate", ylab = "depth")
	dev.off()
}